In [185]:
import gurobipy as grb

In [186]:
import random

In [187]:
# declare n , m here

# Number of students
n = 100

# Number of projects
m = 20

In [188]:
# Weights in the objective function

alpha = 0.5
beta = 0.5

In [189]:
def create_friend_groups(n,m):
    G = []
    for a1 in range(n):
        temp = [0]*n
        for a2 in range(n):
            if(a1//5 == a2//5):
                temp[a2] = 1
        G.append(temp)
    return G

def friend_groups_have_same_project(n,m):
  H = []
  for a in range(n):
    temp = [0]*m
    temp[a//5] = 1
    H.append(temp)
  return H

In [190]:
# Generate the social network matrix

G = []
for i in range(n):
  randomlist = []
  for j in range(n):
    if i==j:
      randomlist.append(0)
    else:
      randomlist.append(random.choice([-1,0,1]))
  G.append(randomlist)
# G = create_friend_groups(n,m)

In [191]:
# Generate the project preference matrix

H = []
for i in range(n):
  randomlist = []
  for j in range(m):
    rand_int = random.randint(0,1)
    randomlist.append(rand_int)
  H.append(randomlist)
# H = friend_groups_have_same_project(n, m)

In [192]:
# Generate the project list input

# P = []

# while(1):
#   P = []
#   temp2 = 0
#   for i in range(m):
#     # t1 is the upper bound for the number of teams that can work on that project
#     # t3 is the upper bound for the team size of any of those teams
#     (t1,t3) = (random.randint(1,3), random.randint(3,6))
#     temp2 += t1*t3
#     P.append((t1,t3))
#   if(temp2 >= n):
#     break

P = [(1,5)]*m

# P = [(1,2), (1,3), (1,3)]

print(P)

[(1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5)]


In [193]:
GRB = grb.GRB
import pandas as pd
Expert_Skill={

 (0,"AI"):1,(0,"DB"):1,(0,"ML"):0,(0,"Python"):0,(0,"IR"):0,
 (1,"AI"):1,(1,"DB"):1,(1,"ML"):1,(1,"Python"):0,(1,"IR"):0,
 (2,"AI"):1,(2,"DB"):1,(2,"ML"):1,(2,"Python"):0,(2,"IR"):1,
 (3,"AI"):1,(3,"DB"):0,(3,"ML"):1,(3,"Python"):0,(3,"IR"):1,
 }
expert_costs={0:150,1:120,2:120,3:110,4:200}
expert_capacity={0:3,1:3,2:3,3:3,4:4}
Project_Skills={
1:{"AI":2,"DB":2,"ML":2},2:{"ML":1,"IR":1},3:{"IR":1,"AI":1} }
Project_Profit={1:120,2:110,3:150}
projects=[1,2,3]
Expert_Communication_Cost=[
 [0,5,2,2],
 [5,0,5,2],
 [2,5,0,5],
 [2,2,5,0]
 ]
Experts=[0,1,2,3]
Skills=[ "AI","DB","ML","IR","Python"]

In [194]:

model =grb.Model("Cluster Assignment")

In [195]:
y_indices = []
for i in range(m):
    for t in range(P[i][0]):
        for a in range(n):
            y_indices.append((i,t,a))

# print(y_indices)
print(len(y_indices))

2000


In [196]:
y = model.addVars(y_indices,vtype=GRB.BINARY,name="y")
print(len(y))

2000


In [197]:
l_indices = []
for i in range(m):
    for t in range(P[i][0]):
        for a1 in range(n):
            for a2 in range(n):
                l_indices.append((i,t,a1,a2))

# print(l_indices)
print(len(l_indices))

200000


In [198]:
l = model.addVars(l_indices,vtype=GRB.BINARY,name="l")
print(len(l))

200000


In [199]:
# for a in range(n):
#     temp = 0
#     for i in range(m):
#         for t in range(P[i][0]):
#             temp += y[(i,t,a)]
for a in range(n):
    model.addConstr( 1 == sum(y[(i,t,a)] for i in range (m) for t in range (P[i][0])) )
    #model.addConstr( 0 >= sum(y[(i,t,a)] for i in range (m) for t in range (P[i][0])) )
    

In [200]:
for i in range(m):
    for t in range (P[i][0]):
        for a1 in range(n):
            for a2 in range(n):
                model.addConstr(l[(i,t,a1,a2)]<=y[(i,t,a1)])
                model.addConstr(l[(i,t,a1,a2)]<=y[(i,t,a2)])
                model.addConstr(l[(i,t,a1,a2)]>= y[(i,t,a1)]+y[(i,t,a2)]-1)

In [201]:
for i in range(m):
    for t in range (P[i][0]):
        model.addConstr( P[i][1] >= sum(y[(i,t,a)] for a in range(n)))

In [202]:
model.setObjective(sum((y[(i,t,a)]*H[a][i]) for i in range(m) for t in range (P[i][0]) for a in range (n)) + 
sum((l[(i,t,a1, a2)]*G[a1][a2]) for i in range(m) for t in range (P[i][0]) for a1 in range (n) for a2 in range(n) if (a1!=a2))
,GRB.MAXIMIZE)

In [203]:
model.setParam("Timelimit",900)
model.setParam("NodeMethod",2)
model.setParam("Method",5)

Changed value of parameter Timelimit to 900.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter NodeMethod to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 5
   Prev: -1  Min: -1  Max: 5  Default: -1


In [204]:

model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 600120 rows, 202000 columns and 1402000 nonzeros
Model fingerprint: 0x5e87bd29
Variable types: 0 continuous, 202000 integer (202000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 6000 rows and 2000 columns (presolve time = 5s) ...
Presolve removed 6000 rows and 2000 columns
Presolve time: 7.83s
Presolved: 594120 rows, 200000 columns, 1390000 nonzeros
Variable types: 0 continuous, 200000 integer (200000 binary)
Found heuristic solution: objective 40.0000000

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing first log only...

Presolved: 594120 rows, 200000 columns, 1390000 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.

In [206]:
for v in model.getVars():
     if v.x!=0.0:
        print(v.varName, v.x)
print('Obj:', model.objVal)

y[0,0,10] 1.0
y[0,0,17] 1.0
y[0,0,31] 1.0
y[0,0,48] 1.0
y[0,0,96] 1.0
y[1,0,3] 1.0
y[1,0,28] 1.0
y[1,0,32] 1.0
y[1,0,47] 1.0
y[1,0,62] 1.0
y[2,0,9] 1.0
y[2,0,12] 1.0
y[2,0,29] 1.0
y[2,0,66] 1.0
y[2,0,71] 1.0
y[3,0,40] 1.0
y[3,0,67] 1.0
y[3,0,70] 1.0
y[3,0,84] 1.0
y[3,0,89] 1.0
y[4,0,6] 1.0
y[4,0,41] 1.0
y[4,0,73] 1.0
y[4,0,77] 1.0
y[4,0,91] 1.0
y[5,0,33] 1.0
y[5,0,35] 1.0
y[5,0,43] 1.0
y[5,0,46] 1.0
y[5,0,97] 1.0
y[6,0,7] 1.0
y[6,0,38] 1.0
y[6,0,57] 1.0
y[6,0,65] 1.0
y[6,0,86] 1.0
y[7,0,42] 1.0
y[7,0,59] 1.0
y[7,0,81] 1.0
y[7,0,95] 1.0
y[7,0,98] 1.0
y[8,0,15] 1.0
y[8,0,22] 1.0
y[8,0,76] 1.0
y[8,0,82] 1.0
y[8,0,99] 1.0
y[9,0,5] 1.0
y[9,0,37] 1.0
y[9,0,54] 1.0
y[9,0,60] 1.0
y[9,0,64] 1.0
y[10,0,13] 1.0
y[10,0,24] 1.0
y[10,0,72] 1.0
y[10,0,85] 1.0
y[10,0,88] 1.0
y[11,0,23] 1.0
y[11,0,49] 1.0
y[11,0,87] 1.0
y[11,0,92] 1.0
y[11,0,93] 1.0
y[12,0,20] 1.0
y[12,0,44] 1.0
y[12,0,55] 1.0
y[12,0,63] 1.0
y[12,0,68] 1.0
y[13,0,16] 1.0
y[13,0,39] 1.0
y[13,0,51] 1.0
y[13,0,61] 1.0
y[13,0,80] 1.0
y[14,